In [12]:
# fetch_edit_counts.py

import requests
import pandas as pd
import duckdb
from datetime import datetime

# --- Config ---
project = "uz.wikipedia.org"
base_url = "https://wikimedia.org/api/rest_v1/metrics/edits/aggregate"
editor_type = "all-editor-types"
page_type = "content"
granularity = "monthly"
start = "20200101"
end = "20240101"
DUCKDB_PATH = ""

# --- Construct URL ---
url = f"{base_url}/{project}/{editor_type}/{page_type}/{granularity}/{start}/{end}"

response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"API Error: {response.status_code} - {response.text}")

data = response.json()
edit_counts = data["items"][0]["results"]

# --- Convert to DataFrame ---
df = pd.DataFrame(edit_counts)
df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)

df['project'] = project
df.rename(columns={'edits': 'edit_count'}, inplace=True)

df


,timestamp,edit_count,project
0,2020-01-01 00:00:00+00:00,1673,uz.wikipedia.org
1,2020-02-01 00:00:00+00:00,3103,uz.wikipedia.org
2,2020-03-01 00:00:00+00:00,5267,uz.wikipedia.org
3,2020-04-01 00:00:00+00:00,5757,uz.wikipedia.org
4,2020-05-01 00:00:00+00:00,3507,uz.wikipedia.org
5,2020-06-01 00:00:00+00:00,3420,uz.wikipedia.org
6,2020-07-01 00:00:00+00:00,3230,uz.wikipedia.org
7,2020-08-01 00:00:00+00:00,4448,uz.wikipedia.org
8,2020-09-01 00:00:00+00:00,4469,uz.wikipedia.org
9,2020-10-01 00:00:00+00:00,5127,uz.wikipedia.org


In [10]:
print(df.dtypes)

timestamp     datetime64[ns, UTC]
edit_count                  int64
project                    object
dtype: object


In [ ]:
# fetch_edit_counts.py

import requests
import pandas as pd

def fetch_edit_counts(
    project: str,
    start: str = "20200101",
    end: str = "20240101",
    editor_type: str = "all-editor-types",
    page_type: str = "content",
    granularity: str = "monthly"
) -> pd.DataFrame:
    """
    Fetches edit count data for a given Wikimedia project and returns a DataFrame.

    Parameters:
        project (str): The project domain, e.g., 'uz.wikipedia.org'
        start (str): Start date in YYYYMMDD format
        end (str): End date in YYYYMMDD format
        editor_type (str): Type of editor (default: all-editor-types)
        page_type (str): Page content type (default: content)
        granularity (str): Time granularity (default: monthly)

    Returns:
        pd.DataFrame: A dataframe with columns: timestamp, edit_count, project
    """
    base_url = "https://wikimedia.org/api/rest_v1/metrics/edits/aggregate"
    url = f"{base_url}/{project}/{editor_type}/{page_type}/{granularity}/{start}/{end}"

    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"API Error: {response.status_code} - {response.text}")

    data = response.json()
    edit_counts = data["items"][0]["results"]

    df = pd.DataFrame(edit_counts)
    df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True)
    df.rename(columns={'edits': 'edit_count'}, inplace=True)
    df['project'] = project

    return df


In [ ]:
#from fetch_edit_counts import fetch_edit_counts

df = fetch_edit_counts("uz.wikipedia.org")
print(df.head())
